In [ ]:
# =========================
# HTML TABLE → DATABASE PARSER
# =========================

import os
import re
import sqlite3
from bs4 import BeautifulSoup
from tqdm import tqdm
import unicodedata

# =========================
# CONFIG
# =========================
BASE_DIR = "/content"
HTML_BASE_DIR = os.path.join(BASE_DIR, "html")
DB_PATH = os.path.join(BASE_DIR, "financial_data.db")

# =========================
# UTILS
# =========================

def remove_accents(text):
    """Loại bỏ dấu tiếng Việt để so sánh"""
    nfd = unicodedata.normalize('NFD', text)
    return ''.join(char for char in nfd if unicodedata.category(char) != 'Mn')

def clean_number(text):
    """
    Làm sạch số liệu:
    - GIỮ NGUYÊN dấu ngoặc () vì nó biểu thị số âm trong kế toán
    - Loại bỏ dấu chấm phân cách hàng nghìn
    - Thay dấu phẩy thành dấu chấm thập phân
    - Nếu có khoảng trắng, so sánh độ dài dãy số:
      + Dãy số nào dài hơn thì lấy
      + Nếu bằng nhau, ưu tiên lấy cái trước khoảng trắng
    - Trả về STRING (không phải float) để giữ nguyên format
    """
    if not text or not isinstance(text, str):
        return None

    text = text.strip()

    # Kiểm tra xem có ngoặc không (số âm trong kế toán)
    has_parentheses = '(' in text or ')' in text

    # Loại bỏ các ký tự không phải số/ngoặc/dấu (giữ ngoặc, số, dấu trừ, dấu phẩy, dấu chấm)
    text = re.sub(r'[^\d\-.,\s()()]', '', text)

    # Xử lý khoảng trắng: so sánh độ dài dãy số
    if ' ' in text:
        parts = [p.strip() for p in text.split(' ') if p.strip()]

        # Lọc các phần có chứa số
        number_parts = []
        for part in parts:
            # Làm sạch part tạm thời để đếm (bỏ dấu chấm, phẩy)
            clean_part = part.replace('.', '').replace(',', '').replace('(', '').replace(')', '')
            if re.search(r'\d', clean_part):
                # Đếm số chữ số (không tính dấu chấm, phẩy, trừ, ngoặc)
                digit_count = len(re.findall(r'\d', clean_part))
                number_parts.append((part, digit_count))

        if number_parts:
            # Sắp xếp theo độ dài (giảm dần), nếu bằng nhau thì giữ thứ tự gốc (ưu tiên phần đầu)
            number_parts.sort(key=lambda x: x[1], reverse=True)
            text = number_parts[0][0]
        else:
            # Không có số hợp lệ
            return None

    # Loại bỏ dấu chấm (phân cách hàng nghìn) nhưng GIỮ ngoặc
    text = text.replace('.', '')

    # Thay dấu phẩy thành dấu chấm (phân cách thập phân)
    text = text.replace(',', '.')

    # Loại bỏ khoảng trắng
    text = text.replace(' ', '')

    # Kiểm tra xem có số không
    if not re.search(r'\d', text):
        return None

    # Nếu có ngoặc, giữ nguyên format với ngoặc
    if has_parentheses and ('(' in text or ')' in text):
        # Đảm bảo format đúng: (số)
        # Loại bỏ dấu trừ nếu có (vì ngoặc đã biểu thị số âm)
        text = text.replace('-', '')
        # Chuẩn hóa ngoặc
        if '(' not in text and ')' in text:
            text = '(' + text
        if '(' in text and ')' not in text:
            text = text + ')'
        return text

    # Nếu không có ngoặc, trả về string số
    # Loại bỏ ngoặc nếu có (trường hợp ngoặc bị tách)
    text = text.replace('(', '').replace(')', '')

    # Validate là số hợp lệ
    try:
        float(text)
        return text
    except (ValueError, AttributeError):
        return None

def normalize_header(text):
    """Chuẩn hóa header để so sánh"""
    if not text:
        return ""
    text = text.upper().strip()
    text = remove_accents(text)
    return text

def is_code_column(header):
    """Kiểm tra xem cột có phải là cột Mã số không"""
    normalized = normalize_header(header)
    keywords = ['MS', 'MA', 'MAA']  # MA, MÃ đều thành MAA sau khi bỏ dấu
    return any(kw in normalized for kw in keywords)

def is_indicator_column(header):
    """Kiểm tra xem cột có phải là cột Chỉ tiêu không"""
    normalized = normalize_header(header)
    keywords = [
        'TAI SAN', 'TAISAN',
        'NGUON VON', 'NGUONVON',
        'CHI TIEU', 'CHITIEU',
        'NOI DUNG', 'NOIDUNG',
        'KHOAN MUC', 'KHOANMUC',
        'TEN', 'DIEN GIAI', 'DIENGIAI'
    ]
    return any(kw in normalized for kw in keywords)

def clean_code(text):
    """
    Làm sạch mã số:
    - Bỏ text sau khoảng trắng nếu có
    - Chỉ giữ số và dấu chấm (cho mã số dạng 1.1, 2.3.4)
    """
    if not text or not isinstance(text, str):
        return ""

    text = text.strip()

    # Nếu có khoảng trắng, chỉ lấy phần trước khoảng trắng đầu tiên
    if ' ' in text:
        text = text.split(' ')[0]

    # Chỉ giữ số và dấu chấm
    text = re.sub(r'[^\d.]', '', text)

    return text.strip()

def extract_table_data(html_path):
    """
    Trích xuất dữ liệu từ file HTML
    Returns: list of records hoặc None nếu không đủ điều kiện
    """
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    table = soup.find('table')
    if not table:
        return None

    rows = table.find_all('tr')
    if not rows:
        return None

    # Tìm header row (thường là row đầu tiên hoặc có class header)
    headers = []
    data_start_idx = 0

    for idx, row in enumerate(rows):
        cells = row.find_all('td')
        # Kiểm tra nếu có cell nào có class header
        if any('header' in cell.get('class', []) for cell in cells):
            headers = [cell.get_text(separator=' ', strip=True) for cell in cells]
            data_start_idx = idx + 1
            break

    if not headers:
        # Nếu không tìm thấy header với class, lấy row đầu tiên
        first_row_cells = rows[0].find_all('td')
        headers = [cell.get_text(separator=' ', strip=True) for cell in first_row_cells]
        data_start_idx = 1

    # Xác định các cột
    code_col_idx = None
    indicator_col_idx = None
    year_prev_col_idx = None  # Cột cuối cùng (năm trước)
    year_curr_col_idx = None  # Cột gần cuối (năm sau)

    # Tìm cột Mã số và Chỉ tiêu
    for idx, header in enumerate(headers):
        if code_col_idx is None and is_code_column(header):
            code_col_idx = idx
        if indicator_col_idx is None and is_indicator_column(header):
            indicator_col_idx = idx

    # Xác định cột năm: cột cuối và cột gần cuối
    # Bỏ qua các cột trống ở cuối
    non_empty_cols = []
    for idx, header in enumerate(headers):
        if header.strip() or idx in [code_col_idx, indicator_col_idx]:
            non_empty_cols.append(idx)

    if len(non_empty_cols) >= 2:
        year_prev_col_idx = non_empty_cols[-1]  # Cột cuối cùng = năm trước
        year_curr_col_idx = non_empty_cols[-2]  # Cột gần cuối = năm sau

    # Kiểm tra đủ 3 cột cần thiết
    if code_col_idx is None or year_prev_col_idx is None or year_curr_col_idx is None:
        return None

    # Nếu không có cột chỉ tiêu, có thể sử dụng cột khác (thường là cột đầu tiên)
    if indicator_col_idx is None:
        # Tìm cột đầu tiên không phải là cột mã số
        for idx, header in enumerate(headers):
            if idx != code_col_idx:
                indicator_col_idx = idx
                break

    # Trích xuất dữ liệu từ các hàng
    records = []
    for row in rows[data_start_idx:]:
        cells = row.find_all('td')
        if len(cells) <= max(code_col_idx, indicator_col_idx or 0, year_prev_col_idx, year_curr_col_idx):
            continue

        # Lấy dữ liệu
        code_raw = cells[code_col_idx].get_text(separator=' ', strip=True) if code_col_idx < len(cells) else ""
        indicator = cells[indicator_col_idx].get_text(separator=' ', strip=True) if indicator_col_idx is not None and indicator_col_idx < len(cells) else ""

        year_prev_text = cells[year_prev_col_idx].get_text(separator=' ', strip=True) if year_prev_col_idx < len(cells) else ""
        year_curr_text = cells[year_curr_col_idx].get_text(separator=' ', strip=True) if year_curr_col_idx < len(cells) else ""

        # Làm sạch mã số (bỏ text sau khoảng trắng và ký tự không phải số)
        code = clean_code(code_raw)

        # Làm sạch số liệu
        year_prev = clean_number(year_prev_text)
        year_curr = clean_number(year_curr_text)

        # Kiểm tra nếu năm trước/năm sau trùng với mã số → coi như không có
        if code and year_prev is not None:
            try:
                # So sánh mã số với năm trước
                # Nếu year_prev có ngoặc, bỏ ngoặc để so sánh
                year_prev_compare = year_prev.replace('(', '').replace(')', '')
                code_as_number = float(code)
                year_prev_number = float(year_prev_compare)
                if abs(code_as_number - year_prev_number) < 0.001:
                    year_prev = None
            except (ValueError, TypeError):
                pass  # Nếu code hoặc year không phải số, bỏ qua

        if code and year_curr is not None:
            try:
                # So sánh mã số với năm sau
                year_curr_compare = year_curr.replace('(', '').replace(')', '')
                code_as_number = float(code)
                year_curr_number = float(year_curr_compare)
                if abs(code_as_number - year_curr_number) < 0.001:
                    year_curr = None
            except (ValueError, TypeError):
                pass

        # Bỏ qua nếu không có mã số
        if not code:
            continue

        # Lưu record ngay cả khi cả 2 năm đều None
        records.append({
            'code': code,
            'indicator': indicator.strip(),
            'year_previous': year_prev,
            'year_current': year_curr
        })

    return records if records else None

# =========================
# DATABASE SETUP
# =========================

def init_database():
    """Khởi tạo database và bảng"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS financial_records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            org TEXT NOT NULL,
            pdf_name TEXT NOT NULL,
            source_file TEXT NOT NULL,
            code TEXT NOT NULL,
            indicator TEXT,
            year_previous TEXT,
            year_current TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # Tạo index để tìm kiếm nhanh hơn
    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_org ON financial_records(org)
    ''')

    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_code ON financial_records(code)
    ''')

    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_source ON financial_records(source_file)
    ''')

    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_pdf_name ON financial_records(pdf_name)
    ''')

    conn.commit()
    conn.close()
    print("✅ Database initialized")

def insert_records(records, org, pdf_name, source_file):
    """Chèn records vào database"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    for record in records:
        cursor.execute('''
            INSERT INTO financial_records (org, pdf_name, source_file, code, indicator, year_previous, year_current)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            org,
            pdf_name,
            source_file,
            record['code'],
            record['indicator'],
            record['year_previous'],
            record['year_current']
        ))

    conn.commit()
    conn.close()

# =========================
# MAIN PROCESSING
# =========================

def extract_pdf_name(source_path):
    """
    Trích xuất tên PDF từ đường dẫn
    Input: "DCS_000000014779169_VI_BaoCaoTaiChinh_KiemToan__2022./page008_table_000_vintern.html"
    Output: "DCS_000000014779169_VI_BaoCaoTaiChinh_KiemToan__2022"
    """
    # Lấy thư mục đầu tiên trong path (tên thư mục chứa file)
    parts = source_path.split('/')
    if len(parts) > 0:
        folder_name = parts[0]
        # Loại bỏ dấu chấm ở cuối nếu có (như trong "...2022./")
        pdf_name = folder_name.rstrip('.')
        return pdf_name
    return source_path

def extract_org_code(pdf_name):
    """
    Trích xuất mã tổ chức từ tên PDF
    Input: "VXB_000000013563283_CBTT_BCTC_2023_F"
    Output: "VXB"

    Input: "DCS_000000014779169_VI_BaoCaoTaiChinh_KiemToan__2022"
    Output: "DCS"
    """
    # Lấy phần đầu tiên trước dấu gạch dưới
    parts = pdf_name.split('_')
    if len(parts) > 0:
        return parts[0].strip()
    return pdf_name

def process_all_html_files():
    """Xử lý tất cả các file HTML và lưu vào database"""

    if not os.path.exists(HTML_BASE_DIR):
        print(f"❌ HTML directory not found: {HTML_BASE_DIR}")
        return

    # Khởi tạo database
    init_database()

    total_files = 0
    processed_files = 0
    skipped_files = 0
    total_records = 0

    # Duyệt qua tất cả các thư mục
    for base_name in os.listdir(HTML_BASE_DIR):
        html_dir = os.path.join(HTML_BASE_DIR, base_name)

        if not os.path.isdir(html_dir):
            continue

        print(f"\n📁 Processing: {base_name}")

        # Trích xuất pdf_name và org từ tên thư mục
        pdf_name = base_name.rstrip('.')
        org = extract_org_code(pdf_name)

        html_files = [f for f in os.listdir(html_dir) if f.endswith('.html')]

        for html_file in tqdm(html_files, desc=f"  {base_name}"):
            total_files += 1
            html_path = os.path.join(html_dir, html_file)

            try:
                records = extract_table_data(html_path)

                if records:
                    # source_file sẽ là: "DCS_.../page008_table_000_vintern.html"
                    source_name = f"{base_name}/{html_file}"
                    insert_records(records, org, pdf_name, source_name)
                    processed_files += 1
                    total_records += len(records)
                else:
                    skipped_files += 1

            except Exception as e:
                print(f"    ⚠️  Error processing {html_file}: {str(e)}")
                skipped_files += 1

    print("\n" + "=" * 60)
    print(f"✅ PROCESSING COMPLETE!")
    print(f"📊 Statistics:")
    print(f"   - Total files scanned: {total_files}")
    print(f"   - Successfully processed: {processed_files}")
    print(f"   - Skipped (insufficient data): {skipped_files}")
    print(f"   - Total records inserted: {total_records}")
    print(f"💾 Database saved to: {DB_PATH}")
    print("=" * 60)

# =========================
# RUN
# =========================

if __name__ == "__main__":
    process_all_html_files()

✅ Database initialized

📁 Processing: VNY_000000014707127_3.1_VI_VNY_Bao_cao_tai_chinh_hop_nhat_nam_2024da_nen


  VNY_000000014707127_3.1_VI_VNY_Bao_cao_tai_chinh_hop_nhat_nam_2024da_nen: 100%|██████████| 2/2 [00:00<00:00, 44.62it/s]



📁 Processing: WTC_000000014694194_Bao_cao_TC_2024_sau_KT_ky_so


  WTC_000000014694194_Bao_cao_TC_2024_sau_KT_ky_so: 100%|██████████| 2/2 [00:00<00:00, 40.75it/s]



📁 Processing: SBD_000000015054784_VI_BaoCaoTaiChinhKiemToan_Hop_Nhat_2024


...


📁 Processing: CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_2024


  CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_2024: 100%|██████████| 2/2 [00:00<00:00, 42.70it/s]



📁 Processing: CMC_000000014715923_VI_BaoCaoTaiChinh_KiemToan_2024.


  CMC_000000014715923_VI_BaoCaoTaiChinh_KiemToan_2024.: 100%|██████████| 2/2 [00:00<00:00, 37.80it/s]


✅ PROCESSING COMPLETE!
📊 Statistics:
   - Total files scanned: 79
   - Successfully processed: 55
   - Skipped (insufficient data): 24
   - Total records inserted: 1143
💾 Database saved to: /content/financial_data.db


In [ ]:
import zipfile
import os

zip_file_path = '/content/html (10).zip'
extract_dir = '/content/html'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extracted '{zip_file_path}' to '{extract_dir}'")


✅ Extracted '/content/html (10).zip' to '/content/html'


In [ ]:
# List the contents of the extracted directory to verify
import os
print(os.listdir('/content/html'))

['DCS_000000014779169_VI_BaoCaoTaiChinh_KiemToan__2022.', 'CT6_000000014858118_Bao_cao_kiem_toan_tieng_Vietda_nen', 'BVL_000000014725614_VI_BCTC_hop_nhat_2024.signed', 'BT6_000000014970920_VI_BaoCaoTaiChinh_KiemToan_2024', 'APP_000000014774810_BCTC_Kiem_toan_2024', 'CMI_000000014698416_VI_BaoCaoTaiChinh_KiemToan_2024', 'CMC_000000014715923_VI_BaoCaoTaiChinh_KiemToan_2024.']


In [ ]:
# =========================
# QUERY FINANCIAL DATABASE
# =========================

import sqlite3
import pandas as pd
from tabulate import tabulate

DB_PATH = "/content/financial_data.db"

# =========================
# CONNECT TO DATABASE
# =========================

def get_connection():
    """Tạo kết nối đến database"""
    return sqlite3.connect(DB_PATH)

# =========================
# BASIC QUERIES
# =========================

def show_table_info():
    """Hiển thị thông tin bảng"""
    conn = get_connection()
    cursor = conn.cursor()

    # Lấy schema
    cursor.execute("PRAGMA table_info(financial_records)")
    columns = cursor.fetchall()

    print("📊 TABLE SCHEMA: financial_records")
    print("=" * 70)
    for col in columns:
        print(f"  {col[1]:20} {col[2]:10} {'NOT NULL' if col[3] else ''} {'PRIMARY KEY' if col[5] else ''}")

    # Đếm tổng số records
    cursor.execute("SELECT COUNT(*) FROM financial_records")
    total = cursor.fetchone()[0]
    print(f"\n📈 Total Records: {total:,}")

    # Đếm số file nguồn
    cursor.execute("SELECT COUNT(DISTINCT source_file) FROM financial_records")
    files = cursor.fetchone()[0]
    print(f"📁 Unique Source Files: {files:,}")

    # Đếm số mã số unique
    cursor.execute("SELECT COUNT(DISTINCT code) FROM financial_records")
    codes = cursor.fetchone()[0]
    print(f"🔢 Unique Codes: {codes:,}")

    conn.close()
    print("=" * 70)

def show_sample_records(limit=10):
    """Hiển thị các records mẫu"""
    conn = get_connection()

    query = f"""
        SELECT
            id,
            source_file,
            code,
            indicator,
            year_previous,
            year_current
        FROM financial_records
        LIMIT {limit}
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    print(f"\n📋 SAMPLE RECORDS (First {limit}):")
    print("=" * 120)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

def search_by_code(code):
    """Tìm kiếm theo mã số"""
    conn = get_connection()

    query = """
        SELECT
            id,
            source_file,
            code,
            indicator,
            year_previous,
            year_current
        FROM financial_records
        WHERE code = ?
        ORDER BY source_file
    """

    df = pd.read_sql_query(query, conn, params=(code,))
    conn.close()

    if len(df) == 0:
        print(f"❌ No records found for code: {code}")
        return

    print(f"\n🔍 SEARCH RESULTS for Code: {code}")
    print("=" * 120)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    print(f"\nFound {len(df)} record(s)")

def search_by_indicator(keyword):
    """Tìm kiếm theo chỉ tiêu (tìm kiếm gần đúng)"""
    conn = get_connection()

    query = """
        SELECT
            id,
            source_file,
            code,
            indicator,
            year_previous,
            year_current
        FROM financial_records
        WHERE indicator LIKE ?
        ORDER BY source_file, code
        LIMIT 50
    """

    df = pd.read_sql_query(query, conn, params=(f'%{keyword}%',))
    conn.close()

    if len(df) == 0:
        print(f"❌ No records found for indicator containing: {keyword}")
        return

    print(f"\n🔍 SEARCH RESULTS for Indicator: '{keyword}'")
    print("=" * 120)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    print(f"\nFound {len(df)} record(s) (limited to 50)")

def get_records_by_source(source_file):
    """Lấy tất cả records từ một file nguồn"""
    conn = get_connection()

    query = """
        SELECT
            id,
            code,
            indicator,
            year_previous,
            year_current
        FROM financial_records
        WHERE source_file = ?
        ORDER BY id
    """

    df = pd.read_sql_query(query, conn, params=(source_file,))
    conn.close()

    if len(df) == 0:
        print(f"❌ No records found for source: {source_file}")
        return

    print(f"\n📄 RECORDS from: {source_file}")
    print("=" * 120)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    print(f"\nTotal: {len(df)} record(s)")

def list_all_sources():
    """Liệt kê tất cả các file nguồn"""
    conn = get_connection()

    query = """
        SELECT
            source_file,
            COUNT(*) as record_count,
            MIN(year_previous) as min_year_prev,
            MAX(year_previous) as max_year_prev
        FROM financial_records
        GROUP BY source_file
        ORDER BY source_file
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    print("\n📚 ALL SOURCE FILES:")
    print("=" * 100)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
    print(f"\nTotal: {len(df)} file(s)")

def get_summary_by_code():
    """Thống kê theo mã số"""
    conn = get_connection()

    query = """
        SELECT
            code,
            COUNT(*) as occurrences,
            COUNT(DISTINCT source_file) as file_count,
            AVG(year_previous) as avg_year_prev,
            AVG(year_current) as avg_year_curr
        FROM financial_records
        GROUP BY code
        HAVING occurrences > 1
        ORDER BY occurrences DESC
        LIMIT 30
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    print("\n📊 SUMMARY BY CODE (Top 30 most common):")
    print("=" * 100)
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False, floatfmt='.2f'))

def export_to_csv(output_path="/content/financial_data_export.csv"):
    """Export toàn bộ dữ liệu ra CSV"""
    conn = get_connection()

    query = """
        SELECT
            id,
            source_file,
            code,
            indicator,
            year_previous,
            year_current,
            org,
            created_at
        FROM financial_records
        ORDER BY source_file, id
    """

    df = pd.read_sql_query(query, conn)
    conn.close()

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ Exported {len(df)} records to: {output_path}")


def custom_query(sql):
    """Thực thi câu query tùy chỉnh"""
    conn = get_connection()

    try:
        df = pd.read_sql_query(sql, conn)
        conn.close()

        print("\n🔧 CUSTOM QUERY RESULT:")
        print("=" * 120)
        print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
        print(f"\nReturned {len(df)} row(s)")
        return df
    except Exception as e:
        conn.close()
        print(f"❌ Query Error: {str(e)}")
        return None

# =========================
# INTERACTIVE MENU
# =========================

def show_menu():
    """Hiển thị menu tương tác"""
    print("\n" + "=" * 70)
    print("💾 FINANCIAL DATABASE QUERY TOOL")
    print("=" * 70)
    print("1. Show database info")
    print("2. Show sample records")
    print("3. Search by code")
    print("4. Search by indicator (keyword)")
    print("5. List all source files")
    print("6. Get records from specific source file")
    print("7. Summary by code")
    print("8. Export to CSV")
    print("9. Run custom SQL query")
    print("0. Exit")
    print("=" * 70)

def main():
    """Main interactive function"""
    while True:
        show_menu()
        choice = input("\nEnter your choice: ").strip()

        if choice == '1':
            show_table_info()

        elif choice == '2':
            limit = input("Number of records to show (default 10): ").strip()
            limit = int(limit) if limit.isdigit() else 10
            show_sample_records(limit)

        elif choice == '3':
            code = input("Enter code to search: ").strip()
            if code:
                search_by_code(code)

        elif choice == '4':
            keyword = input("Enter keyword to search in indicator: ").strip()
            if keyword:
                search_by_indicator(keyword)

        elif choice == '5':
            list_all_sources()

        elif choice == '6':
            source = input("Enter source file name: ").strip()
            if source:
                get_records_by_source(source)

        elif choice == '7':
            get_summary_by_code()

        elif choice == '8':
            output = input("Output path (default: /content/financial_data_export.csv): ").strip()
            output = output if output else "/content/financial_data_export.csv"
            export_to_csv(output)

        elif choice == '9':
            print("\nEnter SQL query (press Enter twice to execute):")
            lines = []
            while True:
                line = input()
                if line == "":
                    break
                lines.append(line)
            sql = "\n".join(lines)
            if sql.strip():
                custom_query(sql)

        elif choice == '0':
            print("\n👋 Goodbye!")
            break

        else:
            print("❌ Invalid choice. Please try again.")

        input("\nPress Enter to continue...")

# =========================
# QUICK ACCESS FUNCTIONS
# =========================

# Bạn có thể gọi trực tiếp các function này:

# show_table_info()
# show_sample_records(20)
# search_by_code("100")
# search_by_indicator("Tài sản")
# list_all_sources()
# get_summary_by_code()
# export_to_csv()

# =========================
# RUN
# =========================

if __name__ == "__main__":
    # Chạy menu tương tác
    main()

    # HOẶC gọi trực tiếp các function:
    # show_table_info()
    # show_sample_records(10)


💾 FINANCIAL DATABASE QUERY TOOL
1. Show database info
2. Show sample records
3. Search by code
4. Search by indicator (keyword)
5. List all source files
6. Get records from specific source file
7. Summary by code
8. Export to CSV
9. Run custom SQL query
0. Exit

Enter your choice: 8
Output path (default: /content/financial_data_export.csv): 
✅ Exported 1143 records to: /content/financial_data_export.csv

Press Enter to continue...

💾 FINANCIAL DATABASE QUERY TOOL
1. Show database info
2. Show sample records
3. Search by code
4. Search by indicator (keyword)
5. List all source files
6. Get records from specific source file
7. Summary by code
8. Export to CSV
9. Run custom SQL query
0. Exit


In [ ]:
import os
import shutil

directory_to_remove = '/content/html/html (10)/VXB_000000013563283_CBTT_BCTC_2023_F'

if os.path.exists(directory_to_remove):
    shutil.rmtree(directory_to_remove)
    print(f"✅ Thư mục '{directory_to_remove}' đã được xóa thành công.")
else:
    print(f"❌ Thư mục '{directory_to_remove}' không tồn tại.")


✅ Thư mục '/content/html/html (10)/VXB_000000013563283_CBTT_BCTC_2023_F' đã được xóa thành công.
